### Imports

In [ ]:
import pandas as pd
import numpy as np
import torch as th
from torch.utils.data import DataLoader, TensorDataset, Dataset
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import lightning as L
import seaborn as sns
# from torchsummary import summary
from torchinfo import summary
## Settings
plt.rcParams['figure.figsize'] = [20, 7]
from tqdm.notebook import tqdm
tqdm.pandas()

### Preliminary Analysis

In [ ]:
df = pd.read_csv("../data/Electricity_production.csv")
# df.index.freq = 'MS'

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = (
    df
    .assign(date=lambda x: pd.to_datetime(x["DATE"]))
    .assign(month=lambda x: x["date"].dt.month,
            year=lambda x: x["date"].dt.year)
    .set_index("date")
    .drop("DATE", axis=1)
    .rename(columns={"IPG2211A2N":"production"})
)
df.head()

In [ ]:
df.plot(y="production")

In [ ]:
results = seasonal_decompose(df['production'])
results.plot()
plt.show()

In [ ]:
pd.plotting.autocorrelation_plot(df['production'])

In [ ]:
plot_acf(df['production'])
plt.show()

In [ ]:
plot_pacf(df['production'])
plt.show()

### Data Prep

In [ ]:
test_indices = 100
train_indices = df.shape[0]- test_indices
train_indices, test_indices

In [ ]:
train = df['production'].values[:train_indices]
val = df['production'].values[train_indices:train_indices+50]
test = df['production'].values[train_indices+50:]
train.shape, val.shape, test.shape

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train.reshape(-1, 1))

In [ ]:
scaler.data_max_

In [ ]:
scaler.data_min_

In [ ]:
train.shape

In [ ]:
train = scaler.transform(train.reshape(-1, 1))
val = scaler.transform(val.reshape(-1, 1))
test = scaler.transform(test.reshape(-1, 1))

In [ ]:
train.shape

In [ ]:
# # create sequence data
# def create_sequence(input_data, sequence_length):
#     sequences = []
#     data_size = len(input_data)
#     for i in range(data_size - sequence_length):
#         features = input_data[i: i + sequence_length]
#         target = input_data[i + sequence_length]
#         sequences.append((th.Tensor(features), th.Tensor(target)))
#     return sequences

# sequence_length = 6
# train_sequences = create_sequence(train, sequence_length)
# test_sequences = create_sequence(test, sequence_length)
# (len(train_sequences), len(test_sequences))

### Pytorch Dataset and DataModule

In [ ]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, x: np.ndarray , sequence_length: int, device: str="cpu"):
        self.x = x
        self.sequence_length = sequence_length
        self.device=device

    def __len__(self):
        return len(self.x) - (self.sequence_length)
        
    def __getitem__(self, idx):
        return (th.Tensor(self.x[idx: idx+self.sequence_length]), th.Tensor(self.x[idx+self.sequence_length]))

In [ ]:
class ElectricityDataModule(L.LightningDataModule):
    def __init__(self, 
                 train_sequences: np.ndarray, 
                 test_sequences:np.ndarray, 
                 val_sequences: np.ndarray,
                 sequence_length: int, 
                 batch_size: int=10):
        super().__init__()
        self.train_sequences    =   train_sequences
        self.test_sequences     =   test_sequences
        self.val_sequences      =   val_sequences
        self.sequence_length    =   sequence_length
        self.batch_size         =   batch_size
        
    def setup(self, stage=None):
        self.train_dataset  =   TimeSeriesDataset(self.train_sequences, self.sequence_length)
        self.val_dataset    =   TimeSeriesDataset(self.val_sequences, self.sequence_length)
        self.test_dataset   =   TimeSeriesDataset(self.test_sequences, self.sequence_length)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )

    def predict_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )

In [ ]:
test.shape

### Pytorch Model

In [ ]:
# Creating the LSTM model
class LSTMModel(L.LightningModule):
    def __init__(self, n_features, n_hidden=128, n_layers=2, lr=1e-3, dropout=0.2):
        super().__init__()
        self.n_hidden = n_hidden
        self.lstm = th.nn.LSTM(input_size = n_features, hidden_size=n_hidden, num_layers=n_layers, batch_first=True)
        self.output_layer = th.nn.Linear(n_hidden, 1)
        self.layer_norm = th.nn.LayerNorm(n_hidden)
        self.loss = th.nn.MSELoss()
        self.learning_rate = lr

    def forward(self, x):
        # self.lstm.flatten_parameters()
        lstm_out, _ = self.lstm(x)
        lstm_out_ = self.layer_norm(lstm_out[:, -1])
        output = self.output_layer(lstm_out_)
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        # print(x.shape)
        output = self(x)
        loss = F.mse_loss(output, y)
        self.log(f"train_loss", loss, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        x_hat = self(x)
        test_loss = F.mse_loss(x_hat, y)
        self.log("test_loss", test_loss, logger=True)
    
    def validation_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        x_hat = self(x)
        test_loss = F.mse_loss(x_hat, y)
        self.log("validation_loss", test_loss, logger=True)

    def predict_step(self, batch, batch_idx):
        x, y = batch
        return self(x)

    def configure_optimizers(self):
        optimiser = th.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimiser

### Training & Testing

In [ ]:
## BEST TRAINING PARAMS
N_EPOCHS = 500
BATCH_SIZE = 8
SEQUENCE_LENGTH = 5
N_HIDDEN = 32
N_LAYERS = 1
LR=1e-4

# N_EPOCHS = 500
# BATCH_SIZE = 8
# SEQUENCE_LENGTH = 6
# N_HIDDEN = 32
# N_LAYERS = 1
# LR=1e-4

In [ ]:
data_module = ElectricityDataModule(train_sequences=train, test_sequences=test,val_sequences=val, sequence_length=SEQUENCE_LENGTH, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = LSTMModel(n_features=1, n_hidden=N_HIDDEN, n_layers=N_LAYERS, lr=LR)
print(model)

In [ ]:
summary(model, input_size=(1,5,1))

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ./lightning_logs/

In [ ]:
checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpont",
    save_top_k=1,
    # verbose=True,
    monitor="validation_loss",
    mode="min"
)
logger = L.pytorch.loggers.tensorboard.TensorBoardLogger(
    "lightning_logs",
    name="electricity-prediction"
)

early_stopping_callbacks = L.pytorch.callbacks.EarlyStopping(
    monitor="train_loss",
    patience=5
)

trainer = L.Trainer(
    logger=logger,
    callbacks=[early_stopping_callbacks, checkpoint_callback],
    max_epochs=N_EPOCHS,
    # enable_progress_bar=True
)

In [ ]:
trainer.fit(model=model, datamodule=data_module)

In [ ]:
trainer.test(model=model, datamodule=data_module)

In [ ]:
trainer.validate(model=model, datamodule=data_module)

In [ ]:
prediction = trainer.predict(model=model, datamodule=data_module)

In [ ]:
final = []
for temp in prediction:
    final.extend(temp.reshape(1, -1).squeeze(0).tolist())
final_scaled_prediction = scaler.inverse_transform(np.fromiter(final, dtype=np.float32).reshape(-1, 1))

# scaler.inverse_transform(test).shape
actual_final = []
for ele in data_module.predict_dataloader():
    actual_final.extend(ele[1].squeeze(1).tolist())

actual_final = np.fromiter(actual_final, dtype=np.float32)
actual_final = scaler.inverse_transform(actual_final.reshape(-1, 1))
actual_final = actual_final.squeeze(1).tolist()
final_scaled_prediction = final_scaled_prediction.squeeze(1).tolist()
temp = pd.DataFrame(zip(actual_final, final_scaled_prediction), columns=["test", "y_pred"])

#### Output

In [ ]:
_, ax = plt.subplots(1, 1)
sns.lineplot( data=temp, y="test", x=temp.index, ax=ax, label="Actual")
sns.scatterplot( data=temp, y="test", x=temp.index, ax=ax)
sns.lineplot( data=temp, y="y_pred", x=temp.index, ax=ax, label="Predicted")
sns.scatterplot( data=temp, y="y_pred", x=temp.index, ax=ax)
plt.show()

In [ ]:
model.eval()
onnx_program = th.onnx.export(
    model, 
    th.zeros(1, 5, 1),
    "../data/TimeSeries_LSTM.onnx",
    export_params=True
)

![alt text][def]

[def]: ../data/TimeSeries_LSTM.onnx.png "Title1"